# Random forest

random forest regressor: multiple inputs - multiple outputs 
- input
  - user: Orig, Dest, 'depDay', 'arrDay'
  - other attributes: distSeg1, distSeg2, distSeg3, segn, elaptime, detour, market_share, con1, con2, depDay2, depDay3, real_dist
- output: pax, TOT_pax, cluster, dep_hour, dep_min, arr_hour, arr_min, stops, con_time

Don't think about whether it's direct or not at the moment! Go simple!

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### 1. load data

In [58]:
# read the preprocessed data
df = pd.read_csv('C:/GitHub/pdsp2023/backend/dataset/dataprep_v2.csv').drop(columns=['Unnamed: 0'])
display(df.head())

,Orig,con1,con2,Dest,depDay2,depDay3,depDay,elaptime,detour,arrDay,...,stops_1.0,stops_2.0,segn_1.0,segn_2.0,segn_3.0,real_dist,dep_hour,dep_min,arr_hour,arr_min
0,26,64,103,181,2.0,NaN,2.0,535.0,1.02676,3.0,...,1,0,0,1,0,3899.12,15,45,1,40
1,26,64,103,181,4.0,NaN,4.0,535.0,1.02676,5.0,...,1,0,0,1,0,3899.12,15,45,1,40
2,26,64,103,181,5.0,NaN,4.0,775.0,1.02676,5.0,...,1,0,0,1,0,3899.12,15,45,5,40
3,26,64,103,181,5.0,NaN,5.0,535.0,1.02676,6.0,...,1,0,0,1,0,3899.12,15,45,1,40
4,26,64,103,181,6.0,NaN,6.0,535.0,1.02676,7.0,...,1,0,0,1,0,3899.12,15,45,1,40


In [59]:
# read the first preprocessed data & original dataset
df_v1 = pd.read_csv('C:/GitHub/pdsp2023/backend/dataset/dataprep_v1.csv').drop(columns='Unnamed: 0')
df_org = pd.read_csv('C:/GitHub/pdsp2023/backend/dataset/data_org.csv').drop(columns='Unnamed: 0')

In [60]:
# calculate the total waiting time for connection
df_org.columns

Index(['Orig', 'con1_arr', 'con1', 'con2_arr', 'con2', 'Dest', 'mkt_airline1',
       'mkt_flight1', 'op_airline1', 'op_flight1', 'depDay1', 'd_time_1',
       'a_time_1', 'depTime_U1', 'arrTime_U1', 'acrt1', 'distSeg1',
       'mkt_airline2', 'mkt_flight2', 'op_airline2', 'op_flight2', 'depDay2',
       'd_time_2', 'a_time_2', 'depTime_U2', 'arrTime_U2', 'acrt2', 'distSeg2',
       'mkt_airline3', 'mkt_flight3', 'op_airline3', 'op_flight3', 'depDay3',
       'd_time_3', 'a_time_3', 'depTime_U3', 'arrTime_U3', 'acrt3', 'distSeg3',
       'dist', 'segn', 'week', 'depDay', 'deptime', 'deptime_U', 'arrtime',
       'arrtime_U', 'elaptime', 'detour', 'arrDay', 'stops', 'dmcc', 'nLegs',
       'paxe', 'cluster', 'OnD', 'TOT_pax', 'market_share'],
      dtype='object')

In [67]:
# df_org['depTime_U2'] = pd.to_datetime(df_org['depTime_U2'])
# df_org['arrTime_U1'] = pd.to_datetime(df_org['arrTime_U1'])
# df_org['depTime_U3'] = pd.to_datetime(df_org['depTime_U3'])
# df_org['arrTime_U2'] = pd.to_datetime(df_org['arrTime_U2'])

# time for connection
# df['con_time'] = (df_org['depTime_U2']- df_org['arrTime_U1']) + (df_org['depTime_U3'] - df_org['arrTime_U2'])
df['con_time'] = df['con_time'].fillna(0)
df['con_time']

0                       0
1                       0
2                       0
3                       0
4                       0
               ...       
522801                  0
522802                  0
522803                  0
522804    0 days 03:37:00
522805    0 days 04:18:00
Name: con_time, Length: 522806, dtype: object

In [62]:
print(df.columns)

Index(['Orig', 'con1', 'con2', 'Dest', 'depDay2', 'depDay3', 'depDay',
       'elaptime', 'detour', 'arrDay', 'paxe', 'cluster', 'TOT_pax',
       'market_share', 'is_direct_flight', 'stops_0.0', 'stops_1.0',
       'stops_2.0', 'segn_1.0', 'segn_2.0', 'segn_3.0', 'real_dist',
       'dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'con_time'],
      dtype='object')


`A simpler model`

In [63]:
# # of connections & how much time we should wait for connecting flights
X = df.loc[:, ['Orig', 'con1', 'con2', 'Dest', 'depDay2', 'depDay3', 'depDay',
       'elaptime', 'detour', 'arrDay', 'cluster',
       'market_share', 'is_direct_flight', 'stops_0.0', 'stops_1.0',
       'stops_2.0', 'segn_1.0', 'segn_2.0', 'segn_3.0', 'real_dist',
       'dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'con_time']]

X.head()

,Orig,con1,con2,Dest,depDay2,depDay3,depDay,elaptime,detour,arrDay,...,stops_2.0,segn_1.0,segn_2.0,segn_3.0,real_dist,dep_hour,dep_min,arr_hour,arr_min,con_time
0,26,64,103,181,2.0,NaN,2.0,535.0,1.02676,3.0,...,0,0,1,0,3899.12,15,45,1,40,NaT
1,26,64,103,181,4.0,NaN,4.0,535.0,1.02676,5.0,...,0,0,1,0,3899.12,15,45,1,40,NaT
2,26,64,103,181,5.0,NaN,4.0,775.0,1.02676,5.0,...,0,0,1,0,3899.12,15,45,5,40,NaT
3,26,64,103,181,5.0,NaN,5.0,535.0,1.02676,6.0,...,0,0,1,0,3899.12,15,45,1,40,NaT
4,26,64,103,181,6.0,NaN,6.0,535.0,1.02676,7.0,...,0,0,1,0,3899.12,15,45,1,40,NaT


In [64]:
# Didn't include
#  'TOT_pax',
y = df.loc[:, 'paxe']
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: paxe, dtype: float64

1.2 Splitting the dataset into training/test set
https://builtin.com/data-science/random-forest-python<br/>
https://machinelearningmastery.com/random-forest-ensemble-in-python/

In [65]:
from sklearn.model_selection import train_test_split
# random_state: controls the shuffling applied to the data before applying the split. 
# Pass an int for reproducible output across multiple function calls.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# check the size of the splitted dataset
print("Shape of X_train:", X_train.shape) # expected output (418244, 22)
print("Shape of y_train:", y_train.shape) # expected output (418244,)
print("Shape of X_test:", X_test.shape) # expected output (104562, 22)
print("Shape of y_test:", y_test.shape) # expected output (104562,)

Shape of X_train: (418244, 25)
Shape of y_train: (418244,)
Shape of X_test: (104562, 25)
Shape of y_test: (104562,)


In [66]:
from sklearn.impute import SimpleImputer, KNNImputer

# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp = imp.fit(X_train)
# X_train = imp.transform(X_train)

imputer = KNNImputer(n_neighbors=2, weights='uniform')
X_train = imputer.fit_transform(X_train)

# Standardisation
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

TypeError: The DType <class 'numpy.dtype[timedelta64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 

### 3. Creating a random forest regression model and fitting it to the training data

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#### Later, can run a loop to find the best n_estimators ####
# Create a random forest regressor
forest = RandomForestRegressor(n_estimators=20, random_state=0)

# Fit the model to the training data
forest.fit(X_train, y_train)

In [ ]:
# Predict the target variable on the training data
y_train_pred = forest.predict(X_train)

# Predict the target variable on the test data
y_test_pred = forest.predict(X_test)


# Evaluate Model
from sklearn.metrics import mean_squared_error

mse_rf_train = mean_squared_error(y_train, y_train_pred)
mse_rf_test = mean_squared_error(y_test, y_test_pred)

# Report the result
print('Random Forest')
print('--------------------------------------------------------------------------------\n')
print('MSE (Training) = %.4f' % mse_rf_train)
print('MSE (Testing)  = %.4f' % mse_rf_test)

In [ ]:
print(y_test[:10])
print(y_test_pred[:10])

#### Hyperparameter tuning

In [ ]:
from pprint import pprint
# Parameters in use for basic model above
print('Parameters currently in use:\n')
pprint(forest.get_params())

#### Random hyperparameter grid

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features}
               #'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
               #'bootstrap': bootstrap}

pprint(random_grid)

#### Random search training

In [ ]:
# activate GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print('Found GPU at: {}'.format(device_name))
else:
  print('GPU device not found')

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 3 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf,
                               param_distributions = random_grid, 
                               n_iter = 3, cv = 3, verbose=2, 
                               random_state=101, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
# View the best parameters from fitting the random search:
rf_random.best_params_

### 4. Validate the model

#### evaluate the random search

In [ ]:
# from sklearn.metrics import average_precision_score, accuracy_score

def evaluate(model, X_test, y_test):
  y_pred = model.predict(X_test)
  error = np.mean((y_pred-y_test) / y_test) * 100
  accuracy = 100 - error
  print('Model Performance')
  print('Average Error: {:0.4f} degrees.'.format(error))
  print('Accuracy = {:0.2f}%.'.format(accuracy))
    
  return accuracy

# Base Model Performance    
base_model = RandomForestRegressor(random_state=101, n_estimators=10)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
# Random Search Model performance
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
print(forest.score(X_test, y_test))

### 5. Save model

In [ ]:
# import numpy as np

# plt.scatter(X_test, y_test)
# plt.scatter(X_test, y_test, color='red')
# plt.plot(X_test, forest.predict(X_test))
# plt.title('Traffic')

### 6. Predict pax for test data

split dataset

standardisation

prediction on new data

In [ ]:
# Create a Random Forest Regressor
#rf = RandomForestRegressor(random_state=101, 
                           #n_estimators=1800)

# Fit the Random Forest Regressor to the training data
#rf.fit(X_train_std, y_train)

# get the predicted y values for the test set
rf_pred_test_new = rf.predict(X_test_new_std)

print("Random Forest predicted values for X_test:", rf_pred_test_new)

# get the id column from the original dataframe
id_column = rentCH_test['id'].reset_index(drop=True)

# create dataframes for predicted values with id column
df_rf = pd.concat([id_column, pd.DataFrame(rf_pred_test_new, columns=["Random Forest Predictions"])], axis=1)

# save dataframes as CSV files
df_rf.to_csv("rf_predictions.csv", index=False)

## Try out multioutput regressor
https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html#sklearn.multioutput.MultiOutputRegressor
https://scikit-learn.org/stable/auto_examples/ensemble/plot_random_forest_regression_multioutput.html

In [ ]:
print(X.shape[0] * 0.8)
print(X.shape[0] * 0.2)
print(418245+104561 == X.shape[0])

In [ ]:
Xrnd = np.sort(200 * np.random.rand(600, 1) - 100, axis=0)
yrnd = np.array([np.pi * np.sin(Xrnd).ravel(), np.pi * np.cos(Xrnd).ravel()]).T
yrnd += 0.5 - np.random.rand(*yrnd.shape)
print(Xrnd.shape)
print(yrnd.shape)

In [ ]:
### 
# input: dep_date, arr_date
# output: dep_hour, dep_min, arr_hour, arr_min, pax, elapsetime, real_dist

X_multirf = df
y_multirf = df.loc[:, ['real_dist', 'dep_date', 'dep_hour', 'dep_min', 
                   'arr_date', 'arr_hour', 'arr_min']]
Xtrain, Xtest, ytrain, ytest = train_test_split(X_multirf, y_multirf, 
                                                train_size=418245, test_size=104561, random_state=0)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor

# Create a random dataset
max_depth = 30
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=0)
)
regr_multirf.fit(Xtrain, ytrain)

# Predict on new data
y_multirf = regr_multirf.predict(Xtest)
y_multirf